In [2]:
# if you need to install Pybullet:
#   pip3 install Pybullet   (python3)
#   pip install PyBullet    (python2.x)

# let's start with imports
import numpy as np
from numpy.random import seed
from numpy.random import rand

# from engine import *

# seed random number generator
seed(1)

Note: you may need to restart the kernel to use updated packages.


In [2]:
# create a pybullet engine from the engine module
engine = PyBulletEnvironment()


In [ ]:
#load shovel
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
robot_urdf = './urdf/shovel/shovelFlat.urdf'

# load the robot
engine.open_environment(robot_urdf, startPos, startOrientation)

In [4]:
# load pebbles

# get shovel_link position
shovelLinkState = p.getLinkState(engine.ID[1], 2)

# admissible init positions
minpos = np.asarray(shovelLinkState[0]) - 5*1e-1
maxpos = np.asarray(shovelLinkState[0]) + 5*1e-1

# number of pebbles
pebbleNum = 100

# scatter pebbles
for i in range (pebbleNum):
    
    # generate init pos
    startPos = minpos + (rand(3) * (maxpos - minpos))
    startPos[-1] = 0.2
    
    # set orientation
    startOrientation = p.getQuaternionFromEuler([0,0,0])
    
    # load pebble 
    engine.load_urdf('./urdf/pebbles/pebbles.urdf', startPos, startOrientation)    
    
# simulate
engine.simulate(0.5)

In [ ]:
# here we control the shovel to move to a target position and orientation using velocity control
# the loop controls the velocity of the joint until the target position is reached

# in the next params we define the control mode, the max force on the joint, the target position and velocity

# get the current end effector position
endEffectorPos = p.getLinkState(engine.ID[1], 2)[0]
pathLength = 1
pathCurvature = 5
Nwaypoints = 20

# compute the waypoints and print them
waypoints = engine.get_waypoints(endEffectorPos, pathLength, pathCurvature, Nwaypoints)
engine.draw_path(waypoints)
print('Initial position: ' + str(endEffectorPos))
print('Waypoints:\n' + '\n'.join(str(waypoint) for waypoint in waypoints))


In [ ]:
# control mode    
# mode = p.VELOCITY_CONTROL
mode = p.POSITION_CONTROL  

# set params
engine.control_threshold = 1e-2
engine.Target_velocities = [1e-3, 1e-2, 1e-2]  
engine.maxForce = [1e3, 1e3, 1e3]
engine.Pos_gains = [1e-1, 1e-1, 1e-1]
engine.Vel_gains = [1e-1, 1e-1, 1e-1]

engine.control_waypoints(waypoints, control_mode=mode)